# Personal Finances Tracker
![title](tenor.gif "Header")
##### image is from the movie "Dumb and Dumber", copyright New Line Cinema 

In [1]:
import matplotlib as mpl
import matplotlib.font_manager as font_manager
import matplotlib.pyplot as plt 
import numpy as np
import os
import pandas as pd
import seaborn as sb
import time
import pytz 
import datetime
from datetime import datetime
from matplotlib import style
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.dates import DateFormatter
from pandas import Series, DataFrame, ExcelFile, ExcelWriter
from pylab import *
from pylab import rcParams
style.use('fivethirtyeight')

In [2]:
# % = IPython magic function, line below allows matplotlib images to appear
%matplotlib inline

# chart dimensions in inches (width, height)
rcParams['figure.figsize'] = 20, 12
sb.set_style('whitegrid')

### Load time series data

In [3]:
# update Excel file's name
date = '05_01_20'
address = 'Sample_Data_' + date + '.xlsx'

# create dataframe
df = pd.read_excel(address, sheet_name='Money', index_col='Date', parse_dates=True)
df.tail()

,Car,Checking,Savings,CD,Stocks,Mortgage,S&P 500,"S&P 500 CD Index, Excel"
Date,,,,,,,,
2020-03-06,16650,20000,10008.9,5293.1,262876.021505,111000,2747.00,6457.451810
2020-03-20,16500,20000,10009.0,5293.1,214121.226832,110000,2237.00,5258.580160
2020-04-03,16350,20000,10009.1,5293.1,238258.668375,109000,2488.65,5850.141044
2020-04-17,16200,20000,10009.2,5293.1,275297.090422,108000,2875.00,6758.345087
2020-05-01,16050,20000,10009.3,5293.1,271133.847995,107000,2831.00,6654.913023


### Check for any irregularities

In [4]:
# Identify null values
pd.isnull(df)

,Car,Checking,Savings,CD,Stocks,Mortgage,S&P 500,"S&P 500 CD Index, Excel"
Date,,,,,,,,
2016-10-16,False,False,False,False,False,False,False,False
2016-10-21,False,False,False,False,False,False,False,False
2016-11-06,False,False,False,False,False,False,False,False
2016-11-18,False,False,False,False,False,False,False,False
2016-12-02,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
2020-03-06,False,False,False,False,False,False,False,False
2020-03-20,False,False,False,False,False,False,False,False
2020-04-03,False,False,False,False,False,False,False,False


In [5]:
# Look at the column names
df.columns.values

array(['Car', 'Checking', 'Savings', 'CD', 'Stocks', 'Mortgage',
       'S&P 500', 'S&P 500 CD Index, Excel'], dtype=object)

In [6]:
# find no. of rows, no. of columns
df.shape

(94, 8)

In [7]:
# describe dataframe
df.describe()

,Car,Checking,Savings,CD,Stocks,Mortgage,S&P 500,"S&P 500 CD Index, Excel"
count,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000
mean,23025.000000,11574.680851,10004.650000,5170.233830,255565.833539,153500.000000,2690.464362,6324.551861
std,4091.912756,3363.945681,2.727942,103.527292,28679.588230,27279.418371,290.158662,682.084301
min,16050.000000,8500.000000,10000.000000,5000.000000,196149.467940,107000.000000,2085.000000,4901.269394
25%,19537.500000,10220.000000,10002.325000,5068.300000,232293.703866,130250.000000,2459.000000,5780.441937
50%,23025.000000,10220.000000,10004.650000,5179.495000,258865.471049,153500.000000,2725.500000,6406.911142
75%,26512.500000,10500.000000,10006.975000,5291.875000,274040.897307,176750.000000,2875.000000,6758.345087
max,30000.000000,20000.000000,10009.300000,5293.100000,319371.408731,200000.000000,3338.000000,7846.732487


In [8]:
df.max(axis=0)

Car                         30000.000000
Checking                    20000.000000
Savings                     10009.300000
CD                           5293.100000
Stocks                     319371.408731
Mortgage                   200000.000000
S&P 500                      3338.000000
S&P 500 CD Index, Excel      7846.732487
dtype: float64

In [9]:
# Set the font properties (for use in legend)   
font_path = 'C:\Windows\Fonts\Arial.ttf'
font_prop = font_manager.FontProperties(fname=font_path, size=20)

In [10]:
# Set datetime formatting
from datetime import datetime
chi = pytz.timezone("America/Chicago")
fmt = '%Y-%m-%d %H:%M:%S %Z%z'
now = datetime.now(chi)
time_a = now.strftime(fmt)
time_a

'2020-12-13 17:50:11 CST-0600'

In [11]:
type(time_a)

str

In [12]:
# trim last 9 characters
time_b = time_a[:-9]
time_b

'2020-12-13 17:50:11'

In [13]:
# replace : with - to avoid Errno 22
time_c = time_b.replace(':','-')
time_c

'2020-12-13 17-50-11'

### Produce timeseries charts in PDF format

In [16]:
label_size = 20
mpl.rcParams['xtick.labelsize'] = label_size 
mpl.rcParams['ytick.labelsize'] = label_size 

# Set the font dictionaries (for plot title and axis titles)
title_font = {'fontname':'Arial', 'size':'48', 'color':'black', 'weight':'normal',
              'verticalalignment':'bottom'} # Bottom vertical alignment for more space
axis_font = {'fontname':'Arial', 'size':'20'}

from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime
chi = pytz.timezone("America/Chicago")
fmt = '%Y-%m-%d %H:%M:%S %Z%z'
now = datetime.now(chi)
#print(now.strftime(fmt))

with PdfPages('Finances Tracker ' + time_c + '.pdf') as pdf:
    # Convert values to float
    df=df.astype(float)

    # Cash
    plt.xlabel('Date',**axis_font)
    plt.ylabel('$',**axis_font)
    
    # orignal title without values
    #plt.title('Cash',**title_font)
    
    df['Cash']=df['Checking']+df['Savings'] 
    df['Cash'].plot()
    plt.legend(loc='best', prop=font_prop, numpoints=1)   
    
    # Find latest cash balances, create array from data frame
    c1 = df['Cash'].values
    # find latest value from Mortgage Balance array
    c2 = c1[-1]
    # convert value to an integer then string so it can be used on the chart title
    c3 = int(c2)    
    # add to title
    plt.title('Cash = $' + '{:,.0f}'.format(c3) + '.',**title_font)
   
    axes = plt.axes()
    axes.set_ylim([0, 50000])
        
    pdf.savefig()    
    plt.close()    
    
    # CD
    plt.xlabel('Date',**axis_font)
    plt.ylabel('$',**axis_font)
    df['CD'].plot()
    
    # Calculate S&P 500 change from beginning value
    sp_start_value = 2127
    df['dSP_500_CD'] = df['S&P 500'] - sp_start_value
    # Replace the first value (only NaN in dataframe) with 0
    df = df.fillna(0)    
    
    # CD S&P 500 Index
    df['CD S&P 500 Index, Python'] = (df['dSP_500_CD'] / df['S&P 500'].shift(1))
    df['S&P 500 CD Index, Python']=np.where(df['CD']>5000,
    (1+(df['dSP_500_CD']/2127)) * 5000,0)
    # Replace the first value (only NaN in dataframe) with 0
    df = df.fillna(0)
    # convert 'S&P 500 CD Index' from object to float
    df=df.astype(float)    
    df['S&P 500 CD Index, Python'].plot()
    # Plot Excel version too, to check
    df['S&P 500 CD Index, Excel'].plot()
    
    plt.legend(loc='best', prop=font_prop, numpoints=1)
    
    # Find latest CD balances, create array from data frame
    cd1 = df['CD'].values
    # find latest value from Mortgage Balance array
    cd2 = cd1[-1]
    # convert value to an integer then string so it can be used on the chart title
    cd3 = int(cd2)    
    # add to title
    plt.title('CD = $' + '{:,.0f}'.format(cd3) + '.',**title_font)

    pdf.savefig()    
    plt.close()    
   
    # Net Worth
    df['Net Worth'] = df['Car'] + df['Checking'] + df['Savings']+ df['Stocks']- df['Mortgage']
    plt.xlabel('Date',**axis_font)
    plt.ylabel('$',**axis_font)
    # create a Net Worth array from data frame
    nw = df["Net Worth"].values

    # find last value Total Assets value
    nwl = nw[-1]
    nwli = int(nwl)
    plt.title('Net Worth = $' + '{:,.0f}'.format(nwli) + '.',**title_font)

    
    #plt.legend(loc='best', prop=font_prop, numpoints=1)
    df['Net Worth'].plot()
    
    pdf.savefig()    
    plt.close()
   
os.startfile('Finances Tracker ' + time_c + '.pdf')

<ipython-input-16-0ff6b1afd08c>:41: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  axes = plt.axes()
